In [1]:
!rm -f aclImdb_v1.tar.gz
!rm -Rf aclImdb
!rm -f movie_data.csv

In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2018-09-26 20:22:38--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  10.4MB/s    in 13s     

2018-09-26 20:22:51 (6.28 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
!tar xzf aclImdb_v1.tar.gz

In [4]:
import os

numberOfFiles = 0
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        numberOfFiles += len(os.listdir(path))

print 'Number of files in dataset ' + str(numberOfFiles)

Number of files in dataset 50000


In [5]:
!pip install pyprind
!pip install nltk

In [17]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(numberOfFiles)

labels = {'pos':'1', 'neg':'0'}


df = pd.DataFrame()
for s in ('test', 'train'):
	for l in ('pos', 'neg'):
		path ='./aclImdb/%s/%s' % (s, l)
		
		for file in os.listdir(path):
			with open(os.path.join(path, file), 'r') as infile:
				txt = infile.read()
				df = df.append([['__label__' + labels[l], txt]], ignore_index=True)
			pbar.update()

df.columns = ['label', 'text']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:27


In [18]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

In [19]:
df = pd.read_csv('./movie_data.csv')
df_row_count = len(df.index)
df.head(5)

,label,text
0,__label__1,"I came out of ""Dark Blue World"" feeling (sigh)..."
1,__label__0,"About five years ago, my friend and I went to ..."
2,__label__0,The one sheets and newspaper campaign suggeste...
3,__label__1,The Williams family live on a ranch located in...
4,__label__0,This film wasn't programmed in Italian cinemas...


In [20]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
nltk.download('stopwords')
stop = stopwords.words('english')

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text + ''.join(emoticons).replace('-', '')
    text = tokenizer_porter(text)
    text = [w for w in text if w not in stop]
    text = ' '.join(text)
    return text

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
pbar = pyprind.ProgBar(df_row_count)

for index, row in df.iterrows():
    df.at[index, 'text'] = preprocessor(row['text'])
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:17


In [23]:
df.head(5)

,label,text
0,__label__1,came dark blue world feel sigh sad yet wa defi...
1,__label__0,five year ago friend went video rental store g...
2,__label__0,one sheet newspap campaign suggest often far l...
3,__label__1,william famili live ranch locat middl remot de...
4,__label__0,thi film program italian cinema seen manifest ...


In [36]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [45]:
import csv

train.to_csv('movie_data_train.csv', header = False, sep='\t', index=False, quoting=csv.QUOTE_NONE)
test.to_csv('movie_data_test.csv', header = False, sep='\t', index=False, quoting=csv.QUOTE_NONE)